In [ ]:
!pip install opencv-python opencv-python-headless
!pip install mediapipe
!pip install ultralytics
from ultralytics import YOLO

In [ ]:
#compete model with mediapipe for foot counting and auto skip
import os
import cv2
from ultralytics import YOLO
import mediapipe as mp
import matplotlib.pyplot as plt

# Utility functions
def visualize_image(image, comment):
    """Display an image with a comment."""
    print(comment)
    plt.figure(figsize=(10, 10))
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))  # Convert BGR to RGB for visualization
    plt.axis('off')
    plt.show()

def save_image(image, path, comment=""):
    """Save an image to the specified path."""
    cv2.imwrite(path, image)
    print(f"{comment} {path}")

def detect_person_with_yolo(image):
    """Detect a person in the image using YOLO."""
    try:
        model = YOLO('yolov8n.pt')  # Use YOLOv8 nano model
        results = model(image)
        for result in results[0].boxes.data.tolist():
            x_min, y_min, x_max, y_max, confidence, class_id = result
            if int(class_id) == 0:  # Class "person" in YOLO
                box_width = int(x_max - x_min)
                box_height = int(y_max - y_min)
                return [int(x_min), int(y_min), box_width, box_height]
        return None  # No person found
    except Exception as e:
        print(f"Error finding person with YOLO: {e}")
        return None

def validate_keypoints_with_mediapipe(image, box):
    """Check if the image contains a nose and two feet using MediaPipe and return their coordinates."""
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose(static_image_mode=True)

    x_min, y_min, box_width, box_height = box
    cropped_image = image[y_min:y_min + box_height, x_min:x_min + box_width]
    img_rgb = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2RGB)
    results = pose.process(img_rgb)

    keypoints = {}
    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark
        # Get coordinates for nose and feet
        keypoints['nose'] = landmarks[mp_pose.PoseLandmark.NOSE]
        keypoints['left_foot'] = landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX]
        keypoints['right_foot'] = landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX]
    return keypoints

def draw_keypoints(image, box, keypoints):
    """Draw bounding boxes for keypoints (nose, feet) on the image."""
    x_min, y_min, box_width, box_height = box
    img_copy = image.copy()

    # Draw person bounding box
    cv2.rectangle(img_copy, (x_min, y_min), (x_min + box_width, y_min + box_height), (0, 255, 0), 2)

    # Draw nose and feet if available
    for key in ['nose', 'left_foot', 'right_foot']:
        if key in keypoints:
            kp = keypoints[key]
            kp_x = int(kp.x * box_width) + x_min
            kp_y = int(kp.y * box_height) + y_min
            cv2.rectangle(img_copy, (kp_x - 5, kp_y - 5), (kp_x + 5, kp_y + 5), (0, 0, 255), 2)
            cv2.putText(img_copy, key, (kp_x + 10, kp_y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
    return img_copy

def process_image(input_path, output_folder, working_folder):
    """Process a single image with YOLO and MediaPipe."""
    if not os.path.isfile(input_path):
        print(f"Error: File {input_path} does not exist.")
        return

    os.makedirs(output_folder, exist_ok=True)
    os.makedirs(working_folder, exist_ok=True)

    base_name = os.path.splitext(os.path.basename(input_path))[0]
    img = cv2.imread(input_path)
    if img is None:
        print(f"Error: Unable to read the file {input_path}.")
        return

    person_box = detect_person_with_yolo(img)
    if not person_box:
        print("Error: No person detected.")
        save_image(img, os.path.join(output_folder, f"{base_name}.jpg"), "Saved unprocessed image:")
        return

    keypoints = validate_keypoints_with_mediapipe(img, person_box)
    img_with_keypoints = draw_keypoints(img, person_box, keypoints)

    save_image(img_with_keypoints, os.path.join(working_folder, f"{base_name}_step1_keypoints.jpg"), "Saved image with bounding boxes for keypoints:")

    # Further processing can go here...

def process_images_in_folder(input_folder, output_folder):
    """Process all images in the input folder and organize results."""
    if not os.path.exists(input_folder):
        print(f"Error: Input folder {input_folder} does not exist.")
        return

    working_folder = os.path.join(output_folder, "working_images")
    os.makedirs(working_folder, exist_ok=True)

    for file_name in os.listdir(input_folder):
        input_path = os.path.join(input_folder, file_name)
        if os.path.isfile(input_path):
            process_image(input_path, output_folder, working_folder)

# Main function
process_images_in_folder(INPUT_FOLDER, OUTPUT_FOLDER)

